# Intelligent Crowd Scene Analytics
## Intelligent Sensing Systems Practice Module 2022

## Analytics engine capable of providing persistent monitoring on both indoor and outdoor static CCTVs to provide higher-order insights on crowd social behaviour
## Modules include:
#### - Social Distancing Monitoring (COVID-19)
#### - Crowd Management (Crowd Counting)
#### - Crowd Anomaly (Flow Estimation/Stampede Alert)
#### - Fall Detection (Medical Emergencies)
#### - Violent Behaviour Detection (Fighting, assualt)

### Team: Adriel Kuek, Chua Hao Zi, KC lim & Yap Pow Look

In [1]:
# Import
import torch
from IPython.display import Image, clear_output

clear_output()
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

# limit the number of cpus used by high performance libraries
import os
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["OPENBLAS_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["VECLIB_MAXIMUM_THREADS"] = "1"
os.environ["NUMEXPR_NUM_THREADS"] = "1"

import sys
sys.path.insert(0, './yolov5')

import argparse
import os
import platform
import shutil
import time
from pathlib import Path
import cv2
import torch
import torch.backends.cudnn as cudnn

from yolov5.models.experimental import attempt_load
from yolov5.utils.downloads import attempt_download
from yolov5.models.common import DetectMultiBackend
from yolov5.utils.datasets import LoadImages, LoadStreams
from yolov5.utils.general import (LOGGER, check_img_size, non_max_suppression, scale_coords, 
                                  check_imshow, xyxy2xywh, increment_path)
from yolov5.utils.torch_utils import select_device, time_sync
from yolov5.utils.plots import Annotator, colors
from deep_sort.utils.parser import get_config
from deep_sort.deep_sort import DeepSort

# FILE = Path(__file__).resolve()
ROOT = os.path.abspath('')
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))  # add ROOT to PATH
ROOT = Path(os.path.relpath(ROOT, Path.cwd()))  # relative

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Setup complete. Using torch 1.10.2+cu102 (TITAN Xp)


deep_sort/deep/reid/torchreid/metrics/rank.py:12: UserWarning: Cython evaluation (very fast so highly recommended) is unavailable, now use python evaluation.
  'Cython evaluation (very fast so highly recommended) is '


# Initialise Models
## YOLOv5 + DeepSORT

In [2]:
evaluate = True
half = True
output = 'output/'
imgsz = [640, 640]
device = select_device(device)

# Initialise DeepSORT
cfg = get_config()
cfg.merge_from_file('deep_sort/configs/deep_sort.yaml')
deepsort = DeepSort('osnet_x0_25',
                    device,
                    max_dist=cfg.DEEPSORT.MAX_DIST,
                    max_iou_distance=cfg.DEEPSORT.MAX_IOU_DISTANCE,
                    max_age=cfg.DEEPSORT.MAX_AGE,
                    n_init=cfg.DEEPSORT.N_INIT,
                    nn_budget=cfg.DEEPSORT.NN_BUDGET)

# Initialise half precision - Only for CUDA enabled devices
half &= device.type != 'cpu'

# For MOT16 evaluation - Run multiple inference streams in parallel
if not evaluate:
    if os.path.exists(output):
        pass
        shutil.rmtree(output)
    os.makedirs(output)

# Directories
save_dir = increment_path(Path(ROOT) / 'exp', exist_ok=True)
save_dir.mkdir(parents=True, exist_ok=True)  # make dir

# Load YOLO Model
model = DetectMultiBackend('yolov5/models/crowdhuman_yolov5m.pt',
                            device=device,
                            dnn=True)
stride, names, pt, jit, _ = model.stride, model.names, model.pt, model.jit, model.onnx
imgsz = check_img_size(imgsz, s=stride)

half &= pt and device.type != 'cpu'
if pt:
    model.model.half() if half else model.model.float()

YOLOv5 🚀 2022-2-9 torch 1.10.2+cu102 CUDA:cuda (TITAN Xp, 12195MiB)



Successfully loaded imagenet pretrained weights from "/home/user/.cache/torch/checkpoints/osnet_x0_25_imagenet.pth"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']
Model: osnet_x0_25
- params: 203,568
- flops: 82,316,000


Fusing layers... 
Model Summary: 308 layers, 21041679 parameters, 0 gradients


# Video Extraction Module
## Extract video segment
## Convert video to image frames

In [3]:
show_vid = True

# Video Source
# source = '/media/user/New Volume/MTech/ITSS/WildTrack_dataset/cam1_short.mp4'
source = '/media/user/New Volume/MTech/ITSS/OxfordTownCenter/TownCentre_short.mp4'

# Set Dataloader
vid_path, vid_writer = None, None

# Check if environment supports image displays
if show_vid:
    show_vid = check_imshow()
    print(f'show_vid: {show_vid}')

# Dataloader
dataset = LoadImages(source, img_size=imgsz, stride=stride, auto=pt and not jit)
bs = 1  # batch_size
vid_path, vid_writer = [None] * bs, [None] * bs

show_vid: True


In [4]:
# Get Names and Colours
names = model.module.names if hasattr(model, 'module') else model.names

# extract filename
txt_file_name = source.split('/')[-1].split('.')[0]
txt_path = str(Path(save_dir)) + '/' + txt_file_name + '.txt'

if pt and device.type != 'cpu':
    model(torch.zeros(1, 3, *imgsz).to(device).type_as(next(model.model.parameters())))  # warmup
dt, seen = [0.0, 0.0, 0.0, 0.0], 0

# Configure Detector parameters

In [5]:
# Configuration
augment = True          # Augmented inference
visual = False          # Visualisation function - Set this to false as default
conf_thres = 0.3        # Object confidence threshold
iou_thres = 0.5         # IOU Threshold for NMS
classes = 0             # Filter for class 0 - Person
agnostic_nms = True     # Class agnostic NMS
max_det = 1000          # Max number of detections per image
save_txt = False
save_vid = True

# Social Distance Metrics
## Initialise hyperparameters for social distance model

In [6]:
# Skip this cell
%script false

import socialdistance as sd

# Configuration
dist_thres = 1.0
violation_thres = 5
DepthControlFactor = 0.3
height_control = 1.7
cam = 1

# Global Dictionaries
socialdistance_dict = {}
printed_tracks = []
track_pair_list = []
count = 0
# filter inaccurate yolo bboxs
seen_tracks = {}

# Initialise Model with input configuration
sd = sd.socialdistance(dist_thres, violation_thres, DepthControlFactor, height_control, socialdistance_dict, seen_tracks, count, track_pair_list, printed_tracks)


UsageError: Line magic function `%script` not found (But cell magic `%%script` exists, did you mean that instead?).


# Iterate across images stored in pytorch dataloader and perform detection and tracking
# - Compute Social Distance Violation

In [7]:
# Skip this cell
%script false

with torch.no_grad():
    for frame_idx, (path, img, im0s, vid_cap, s) in enumerate(dataset):

        # Initialise every frame for socialdistance model
        img_centroid = []
        trackid_bbox_centroid = []

        t1 = time_sync()
        img = torch.from_numpy(img).to(device)
        img = img.half() if half else img.float()  # uint8 to fp16/32
        img /= 255.0  # 0 - 255 to 0.0 - 1.0
        if img.ndimension() == 3:
            img = img.unsqueeze(0)
        t2 = time_sync()
        dt[0] += t2 - t1

        # Inference
        visualize = increment_path(save_dir / Path(path).stem, mkdir=True) if visual else False
        pred = model(img, augment=augment, visualize=visualize)
        t3 = time_sync()
        dt[1] += t3 - t2

        # Apply NMS
        pred = non_max_suppression(pred, conf_thres, iou_thres, classes, agnostic_nms, max_det=max_det)
        dt[2] += time_sync() - t3

        # Process detections
        for i, det in enumerate(pred):  # detections per image
            seen += 1
            p, im0, _ = path, im0s.copy(), getattr(dataset, 'frame', 0)

            p = Path(p)  # to Path
            save_path = str(save_dir / p.name)  # im.jpg, vid.mp4, ...
            s += '%gx%g ' % img.shape[2:]  # print string

            # annotator = Annotator(im0, line_width=2, pil=not ascii)

            if det is not None and len(det):
                # Rescale boxes from img_size to im0 size
                det[:, :4] = scale_coords(
                    img.shape[2:], det[:, :4], im0.shape).round()

                # Print results
                for c in det[:, -1].unique():
                    n = (det[:, -1] == c).sum()  # detections per class
                    s += f"{n} {names[int(c)]}{'s' * (n > 1)}, "  # add to string

                xywhs = xyxy2xywh(det[:, 0:4])
                confs = det[:, 4]
                clss = det[:, 5]

                # pass detections to deepsort
                t4 = time_sync()
                outputs = deepsort.update(xywhs.cpu(), confs.cpu(), clss.cpu(), im0)
                t5 = time_sync()
                dt[3] += t5 - t4

                # draw boxes for visualization
                if len(outputs) > 0:
                    for j, (output, conf) in enumerate(zip(outputs, confs)):

                        bboxes = output[0:4]
                        id = output[4]
                        cls = output[5]

                        # Compute box centroid - Social Distance
                        img_centroid = [int(output[0] + (output[2] - output[0])/2), int(output[1]+ (output[3] - output[1])/2)]
                        img_height = output[3] - output[1]

                        # Box definitions
                        x = int(output[0])
                        y = int(output[1])
                        w = int(output[2] - output[0])
                        h = int(output[3] - output[1])
                        bbox_int = x,y,w,h

                        trackid_bbox_centroid.append(['{}'.format(id), '{}'.format(cam), bbox_int, img_centroid, img_height])

                        c = int(cls)  # integer class
                        label = f'ID:{id} ({conf:.2f})'

                        # Comment out this portion and draw our own boxes
                        # annotator.box_label(bboxes, label, color=colors(c, True))

                        if save_txt:
                            # to MOT format
                            bbox_left = output[0]
                            bbox_top = output[1]
                            bbox_w = output[2] - output[0]
                            bbox_h = output[3] - output[1]
                            # Write MOT compliant results to file
                            with open(txt_path, 'a') as f:
                                f.write(('%g ' * 10 + '\n') % (frame_idx + 1, id, bbox_left,  # MOT format
                                                                bbox_top, bbox_w, bbox_h, -1, -1, -1, -1))

                LOGGER.info(f'{s}Done. YOLO:({t3 - t2:.3f}s), DeepSort:({t5 - t4:.3f}s)')

            else:
                deepsort.increment_ages()
                LOGGER.info('No detections')

            # # Stream results
            # im0 = annotator.result()
            # if show_vid:
            #     cv2.imshow(str(p), im0)
            #     if cv2.waitKey(1) == ord('q'):  # q to quit
            #         raise StopIteration

            # Save results (image with detections)
            # if save_vid:
            #     if vid_path != save_path:  # new video
            #         vid_path = save_path
            #         if isinstance(vid_writer, cv2.VideoWriter):
            #             vid_writer.release()  # release previous video writer
            #         if vid_cap:  # video
            #             fps = vid_cap.get(cv2.CAP_PROP_FPS)
            #             w = int(vid_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
            #             h = int(vid_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
            #         else:  # stream
            #             fps, w, h = 30, im0.shape[1], im0.shape[0]

            #         vid_writer = cv2.VideoWriter(save_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))
            #     vid_writer.write(im0)

        img_out = sd.proximity_evaluation(trackid_bbox_centroid, im0, cam)

        # Plot results
        cv2.imshow(str(p), img_out)
        if cv2.waitKey(1) == ord('q'):  # q to quit
            raise StopIteration

        # Save Video
        if save_vid:
            if vid_path != save_path:  # new video
                vid_path = save_path
                if isinstance(vid_writer, cv2.VideoWriter):
                    vid_writer.release()  # release previous video writer
                if vid_cap:  # video
                    fps = vid_cap.get(cv2.CAP_PROP_FPS)
                    w = int(vid_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
                    h = int(vid_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
                else:  # stream
                    fps, w, h = 30, img_out.shape[1], img_out.shape[0]

                vid_writer = cv2.VideoWriter(save_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))
            vid_writer.write(img_out)
    
    vid_cap.release()
    vid_writer.release()
    cv2.destroyAllWindows()

UsageError: Line magic function `%script` not found (But cell magic `%%script` exists, did you mean that instead?).


# Computation for Homography
## - Manual Labour
## - Automated Bird's eye view through CNN

In [8]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# 4 point pairs - p1, p2, p3, p4
points_birdseye = [[400,695],[828,770],[325,1181],[716,1263]]
points_image = [[634,411],[1316,469],[231,717],[1031,847]]
overhead_hmat = np.array([[0.1916320439247891638, 0.7175533976911367962, -0.01133240359600345970], 
                [0.007170477613692259090, 1.414341738339534471, -0.009603207928416342826], 
                [0.000002700896605900791288, 0.0005327386830370514463, 0.1329097656281789108]])

# Get homography matrix through manual annotation
h_mat, status = cv2.findHomography(np.array(points_image), np.array(points_birdseye))
print(f'Homography Matrix: {h_mat}')
print(f'Auto Homography Matrix: {overhead_hmat}')

# Read in image screenshot
img_path = 'homography/OxfordTownCenter.png'
img_cv = cv2.imread(img_path)
img_cv = cv2.cvtColor(img_cv, cv2.COLOR_BGR2RGB)

# Perform perspective warping
target_dim = (1080,1465)
overhead_dim = (2662,2160)

# birdeye_pt = cv2.perspectiveTransform(bbox_pt, h_mat)
warped_image = cv2.warpPerspective(img_cv, h_mat, dsize=target_dim, flags=cv2.INTER_CUBIC)
plt.imshow(warped_image)
plt.show()
# plt.imsave('OxfordBirdsEye_HighRes.jpg', warped_image)

/home/user/anaconda3/envs/itss/lib/python3.7/site-packages/ipykernel_launcher.py:29: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.


Homography Matrix: [[     1.4392      2.1527     -944.03]
 [  -0.027683      6.1859     -1042.5]
 [ 0.00012455   0.0025643           1]]
Auto Homography Matrix: [[    0.19163     0.71755   -0.011332]
 [  0.0071705      1.4143  -0.0096032]
 [ 2.7009e-06  0.00053274     0.13291]]


In [9]:
# Resize image function
def hconcat_resize_min(im_list, interpolation=cv2.INTER_CUBIC):
    h_min = min(im.shape[0] for im in im_list)
    im_list_resize = [cv2.resize(im, (int(im.shape[1] * h_min / im.shape[0]), h_min), interpolation=interpolation)
                      for im in im_list]
    return cv2.hconcat(im_list_resize)

# Run Detector and Tracker, followed byu Homography transformation to birds eye view

In [10]:
import cv2
import numpy as np

# Use Manual h_mat
# height = 1465
# width = 1080

# Use Auto h_mat
height = 2160
width = 2662
scale_resize = 50

with torch.no_grad():
    for frame_idx, (path, img, im0s, vid_cap, s) in enumerate(dataset):

        # Create a blank map canvas every frame
        map_image = np.zeros((height,width,3), np.uint8)

        t1 = time_sync()
        img = torch.from_numpy(img).to(device)
        img = img.half() if half else img.float()  # uint8 to fp16/32
        img /= 255.0  # 0 - 255 to 0.0 - 1.0
        if img.ndimension() == 3:
            img = img.unsqueeze(0)
        t2 = time_sync()
        dt[0] += t2 - t1

        # Inference
        visualize = increment_path(save_dir / Path(path).stem, mkdir=True) if visual else False
        pred = model(img, augment=augment, visualize=visualize)
        t3 = time_sync()
        dt[1] += t3 - t2

        # Apply NMS
        pred = non_max_suppression(pred, conf_thres, iou_thres, classes, agnostic_nms, max_det=max_det)
        dt[2] += time_sync() - t3

        # Process detections
        for i, det in enumerate(pred):  # detections per image
            seen += 1
            p, im0, _ = path, im0s.copy(), getattr(dataset, 'frame', 0)

            p = Path(p)  # to Path
            save_path = str(save_dir / p.name)  # im.jpg, vid.mp4, ...
            s += '%gx%g ' % img.shape[2:]  # print string

            # annotator = Annotator(im0, line_width=2, pil=not ascii)

            if det is not None and len(det):
                # Rescale boxes from img_size to im0 size
                det[:, :4] = scale_coords(
                    img.shape[2:], det[:, :4], im0.shape).round()

                # Print results
                for c in det[:, -1].unique():
                    n = (det[:, -1] == c).sum()  # detections per class
                    s += f"{n} {names[int(c)]}{'s' * (n > 1)}, "  # add to string

                xywhs = xyxy2xywh(det[:, 0:4])
                confs = det[:, 4]
                clss = det[:, 5]

                # pass detections to deepsort
                t4 = time_sync()
                outputs = deepsort.update(xywhs.cpu(), confs.cpu(), clss.cpu(), im0)
                t5 = time_sync()
                dt[3] += t5 - t4

                # draw boxes for visualization
                if len(outputs) > 0:
                    for j, (output, conf) in enumerate(zip(outputs, confs)):

                        bboxes = output[0:4]
                        id = output[4]
                        cls = output[5]

                        # Box definitions
                        x = int(output[0])
                        y = int(output[1])
                        w = int(output[2] - output[0])
                        h = int(output[3] - output[1])
                        bbox_int = x,y,w,h

                        c = int(cls)  # integer class
                        label = f'ID:{id} ({conf:.2f})'

                        # Write Track ID on target base
                        cv2.putText(im0,str(id),(x,y+h+20), 0, 400*1e-3*2, [255,255,255], 2)

                        # Comment out this portion and draw our own boxes
                        # annotator.box_label(bboxes, label, color=colors(c, True))

                        # Compute box base - Homography transformation
                        bbox_pt = np.array([(int(output[0] + (output[2] - output[0])/2), int(output[1]))], dtype='float32')
                        bbox_pt = np.array([bbox_pt], dtype='float32')

                        # Obtain corresponding birdseye point transformation
                        # - manual h_mat
                        # - auto h_mat (overhead_hmat)
                        # birdseye_pt = cv2.perspectiveTransform(bbox_pt, h_mat)
                        birdseye_pt = cv2.perspectiveTransform(bbox_pt, overhead_hmat)                        
                        map_pt = (int(birdseye_pt[0][0][0]), int(birdseye_pt[0][0][1]))

                        # Draw point on 2d plane
                        cv2.circle(map_image, map_pt, 15, (0,0,255), -2)

                        if save_txt:
                            # to MOT format
                            bbox_left = output[0]
                            bbox_top = output[1]
                            bbox_w = output[2] - output[0]
                            bbox_h = output[3] - output[1]
                            # Write MOT compliant results to file
                            with open(txt_path, 'a') as f:
                                f.write(('%g ' * 10 + '\n') % (frame_idx + 1, id, bbox_left,  # MOT format
                                                                bbox_top, bbox_w, bbox_h, -1, -1, -1, -1))

                LOGGER.info(f'{s}Done. YOLO:({t3 - t2:.3f}s), DeepSort:({t5 - t4:.3f}s)')

            else:
                deepsort.increment_ages()
                LOGGER.info('No detections')

            # Plot results by joining the images together
            # scale and resize the image smaller for display
            final_image_joined = hconcat_resize_min([im0, map_image])
            resize_w = int(final_image_joined.shape[1] * scale_resize/100)
            resize_h = int(final_image_joined.shape[0] * scale_resize/100)
            resize_dim = (resize_w, resize_h)
            resized_image = cv2.resize(final_image_joined, resize_dim, interpolation=cv2.INTER_AREA)

            cv2.imshow(str(p), resized_image)
            if cv2.waitKey(1) == ord('q'):  # q to quit
                raise StopIteration

            # # Stream results
            # im0 = annotator.result()
            # if show_vid:
            #     cv2.imshow(str(p), im0)
            #     if cv2.waitKey(1) == ord('q'):  # q to quit
            #         raise StopIteration

        # img_out = sd.proximity_evaluation(trackid_bbox_centroid, im0, cam)

        # # Plot results
        # cv2.imshow(str(p), output_img)
        # if cv2.waitKey(1) == ord('q'):  # q to quit
        #     raise StopIteration

            # Save results (image with detections)
            if save_vid:
                if vid_path != save_path:  # new video
                    vid_path = save_path
                    if isinstance(vid_writer, cv2.VideoWriter):
                        vid_writer.release()  # release previous video writer
                    # if vid_cap:  # video
                    #     fps = vid_cap.get(cv2.CAP_PROP_FPS)
                    #     w = int(vid_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
                    #     h = int(vid_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
                    # else:  # stream
                    fps, w, h = 30, resized_image.shape[1], resized_image.shape[0]

                    # fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
                    vid_writer = cv2.VideoWriter(save_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h), True)
                vid_writer.write(resized_image)
    
    vid_cap.release()
    vid_writer.release()
    cv2.destroyAllWindows()

video 1/1 (1/1523) /media/user/New Volume/MTech/ITSS/OxfordTownCenter/TownCentre_short.mp4: 384x640 36 persons, Done. YOLO:(0.036s), DeepSort:(0.086s)
video 1/1 (2/1523) /media/user/New Volume/MTech/ITSS/OxfordTownCenter/TownCentre_short.mp4: 384x640 32 persons, Done. YOLO:(0.037s), DeepSort:(0.095s)
video 1/1 (3/1523) /media/user/New Volume/MTech/ITSS/OxfordTownCenter/TownCentre_short.mp4: 384x640 31 persons, Done. YOLO:(0.039s), DeepSort:(0.069s)
video 1/1 (4/1523) /media/user/New Volume/MTech/ITSS/OxfordTownCenter/TownCentre_short.mp4: 384x640 32 persons, Done. YOLO:(0.039s), DeepSort:(0.077s)


(1751, 1748)
(1290, 1876)
(1677, 1767)
(1525, 1994)
(1578, 1919)
(1492, 1977)
(2059, 486)
(1986, 1055)
(1428, 602)
(2006, 1188)
(1772, 1588)
(1023, 1025)
(2047, 1201)
(1213, 278)
(1536, 758)
(1323, 592)
(1509, 629)
(1848, 1596)
(1622, 90)
(1539, 2107)
(1189, 983)
(2034, 1016)
(1585, 900)
(1155, 685)
(1563, 58)
(1705, 63)
(1367, 613)
(1538, 2116)
(1658, 2037)
(1754, 1743)
(1289, 1875)
(1675, 1772)
(1526, 1994)
(1577, 1923)
(1490, 1979)
(2050, 507)
(1983, 1055)
(1429, 602)
(2008, 1181)
(1769, 1595)
(1022, 1013)
(2047, 1198)
(1218, 270)
(1535, 758)
(1324, 592)
(1514, 617)
(1866, 1559)
(1624, 100)
(1540, 2107)
(1189, 979)
(2030, 1020)
(1589, 891)
(1155, 703)
(1562, 58)
(1704, 63)
(1369, 606)
(1539, 2117)
(1658, 2037)


video 1/1 (5/1523) /media/user/New Volume/MTech/ITSS/OxfordTownCenter/TownCentre_short.mp4: 384x640 33 persons, Done. YOLO:(0.038s), DeepSort:(0.082s)


(1756, 1739)
(1289, 1873)
(1676, 1771)
(1527, 1995)
(1575, 1927)
(1488, 1981)
(2041, 527)
(1981, 1055)
(1434, 576)
(2012, 1171)
(1764, 1605)
(1023, 1008)
(2048, 1195)
(1222, 279)
(1533, 758)
(1326, 573)
(1518, 604)
(1875, 1539)
(1602, 70)
(1541, 2108)
(1192, 984)
(2027, 1020)
(1592, 891)
(1154, 714)
(1560, 58)
(1702, 63)
(1371, 600)
(1539, 2120)
(1658, 2037)
(1358, 91)
(1621, 90)


video 1/1 (6/1523) /media/user/New Volume/MTech/ITSS/OxfordTownCenter/TownCentre_short.mp4: 384x640 33 persons, Done. YOLO:(0.038s), DeepSort:(0.079s)


...

video 1/1 (943/1523) /media/user/New Volume/MTech/ITSS/OxfordTownCenter/TownCentre_short.mp4: 384x640 19 persons, Done. YOLO:(0.035s), DeepSort:(0.140s)


(1428, 792)
(1908, 71)
(844, 1593)
(1486, 2067)
(1630, 1780)
(1692, 460)
(1498, 757)
(1954, 648)
(1570, 2071)
(1171, 983)
(1190, 1530)
(1461, 793)
(2180, 286)
(1647, 1864)
(2023, 619)
(1378, 2062)
(1194, 487)
(2340, 87)


video 1/1 (944/1523) /media/user/New Volume/MTech/ITSS/OxfordTownCenter/TownCentre_short.mp4: 384x640 18 persons, Done. YOLO:(0.037s), DeepSort:(0.125s)
video 1/1 (945/1523) /media/user/New Volume/MTech/ITSS/OxfordTownCenter/TownCentre_short.mp4: 384x640 19 persons, Done. YOLO:(0.036s), DeepSort:(0.063s)


(1427, 808)
(1906, 71)
(839, 1592)
(1484, 2070)
(1628, 1784)
(1693, 460)
(1497, 757)
(1951, 654)
(1569, 2072)
(1171, 975)
(1192, 1534)
(1460, 798)
(2180, 286)
(1650, 1860)
(2021, 626)
(1379, 2062)
(1195, 487)
(2347, 87)
(1426, 818)
(1903, 71)
(834, 1589)
(1482, 2069)
(1624, 1788)
(1694, 460)
(1494, 768)
(2013, 619)
(1947, 666)
(1568, 2074)
(1171, 966)
(1193, 1534)
(1458, 798)
(2176, 294)
(1652, 1860)
(2015, 638)
(1380, 2061)
(1196, 487)
(2354, 88)


video 1/1 (946/1523) /media/user/New Volume/MTech/ITSS/OxfordTownCenter/TownCentre_short.mp4: 384x640 21 persons, Done. YOLO:(0.034s), DeepSort:(0.089s)
